In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [3]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [15]:
count_vectorizer = feature_extraction.text.CountVectorizer()
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])
example_train_vectors_tfidf = tfidf_vectorizer.fit_transform(train_df["text"][0:5])

In [6]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [16]:
print(example_train_vectors_tfidf[0].todense().shape)
print(example_train_vectors_tfidf[0].todense())

(1, 54)
[[0.         0.         0.         0.23336118 0.28924517 0.23336118
  0.         0.         0.         0.         0.         0.
  0.28924517 0.28924517 0.         0.         0.         0.
  0.28924517 0.         0.         0.         0.         0.
  0.         0.28924517 0.         0.         0.         0.28924517
  0.         0.         0.         0.         0.28924517 0.
  0.         0.         0.         0.28924517 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.28924517 0.23336118 0.         0.28924517 0.        ]]


In [17]:
train_vectors = tfidf_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = tfidf_vectorizer.transform(test_df["text"])

In [18]:
clf = linear_model.RidgeClassifier()

In [19]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.63366337, 0.6122449 , 0.68442211])

In [20]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [21]:
sample_submission = pd.read_csv("./sample_submission.csv")

In [22]:
sample_submission["target"] = clf.predict(test_vectors)

In [23]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [24]:
sample_submission.to_csv("submission.csv", index=False)